# Creating recombined genomes

In [2]:
# Pick random region between two rows in a file
import pandas as pd
from Bio import SeqIO
import numpy as np
import csv

syri_regions = pd.read_csv('../alignment_syri/syri_out/H9920_Bt63_syri.out', delimiter='\t', header=None)

# filtering out just snp regions for now
filtered_syri_regions = syri_regions[syri_regions[10] == 'SNP']
filtered_syri_regions.head()

# Read the genomes from fasta files
genome1 = SeqIO.to_dict(SeqIO.parse("../alignment_syri/genomes/H9920_sorted.fasta", "fasta"))
genome2 = SeqIO.to_dict(SeqIO.parse("../alignment_syri/genomes/Bt63.fasta", "fasta"))

# Define the recombination rate
recombination_rate = 0.5

# Prepare a list to store recombination data
recombination_data = []

# Perform the recombination events for each chromosome
for chrom in genome1.keys():
    # Calculate the number of recombination events
    num_recombination_events = np.random.poisson(recombination_rate)

    for _ in range(num_recombination_events):
        # Filter rows where the first column equals chrom
        filtered_rows = filtered_syri_regions[filtered_syri_regions[0] == chrom]

        # Choose a random row from filtered_rows
        row = filtered_rows.sample(n=1)
        print(row)

# Check if the selected row is the last row
        if row.index[0] != filtered_rows.index[-1]:
            # Look ahead to the next row
            next_row = filtered_rows.loc[row.index[0] + 1]
            
            # Calculate the difference between the values in the next row and the current row for both positions
            position1_diff = round(int(next_row[1]) - int(row.iloc[0, 1]))
            position2_diff = round(int(next_row[6]) - int(row.iloc[0, 6]))

            print(position1_diff, position2_diff)
            # Add the difference to the current position
            position1 = int(row.iloc[0, 1]) + position1_diff
            position2 = int(row.iloc[0, 6]) + position2_diff
            chrom2 = row.iloc[0, 5]

            # Swap the sequences between the two chromosomes
            temp_seq1 = genome1[chrom].seq[:position1] + genome2[chrom2].seq[position2:]
            temp_seq2 = genome2[chrom2].seq[:position2] + genome1[chrom].seq[position1:]

            genome1[chrom].seq = temp_seq1
            genome2[chrom2].seq = temp_seq2

            # Store the recombination data
            recombination_data.append([chrom, position1, chrom2, position2])

# Save the recombined genomes to files
SeqIO.write(genome1.values(), "second_recombined_genome1.fasta", "fasta")
SeqIO.write(genome2.values(), "second_recombined_genome2.fasta", "fasta")

# Save the recombination data to a TSV file
with open("second_recombination_data_test.tsv", "w", newline="") as f:
    writer = csv.writer(f, delimiter="\t")
    writer.writerow(["Chromosome1", "Position1", "Chromosome2", "Position2"])
    writer.writerows(recombination_data)

/home/kwilhoit/miniconda3/envs/cython_env/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3552: DtypeWarning: Columns (1,2) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


          0       1       2  3  4     5       6       7          8     9   \
103314  chr2  684531  684531  C  G  chr2  730950  730950  SNP103287  SYN7   

         10 11  
103314  SNP  -  
2 2
          0      1      2  3  4     5      6      7          8     9    10 11
137278  chr4  96383  96383  A  G  chr4  67967  67967  SNP137239  SYN9  SNP  -
1 1
          0       1       2  3  4     5       6       7          8      9   \
194869  chr7  153292  153292  C  A  chr7  148426  148426  SNP194808  SYN13   

         10 11  
194869  SNP  -  
5 5
          0      1      2  3  4      5      6      7         8     9    10 11
44528  chr11  91239  91239  C  A  chr11  88208  88208  SNP44522  SYN3  SNP  -
20 20
          0       1       2  3  4      5       6       7         8     9   \
88696  chr14  447174  447174  C  G  chr14  454241  454241  SNP88675  SYN6   

        10 11  
88696  SNP  -  
6 6


In [ ]:
%%bash
cat second_recombined_genome1.fasta second_recombined_genome2.fasta > second_concat_recombined.fasta

cat ../alignment_syri/genomes/Bt63.fasta ../alignment_syri/genomes/H9920_sorted.fasta > unrecombined_genome.fasta



# Simulating long reads

In [2]:

%%bash
# Using a sample fastq file from real data to simulate reads (recommended)

pbsim --strategy wgs --method sample --sample /FastData/kwilhoit/recombination_testing/all_barcode02_reads.fastq --depth 20 --genome second_concat_recombined.fasta --prefix wt_second_sample

cat wt_second*.fastq > all_wt_second_sample_simulated_reads.fastq



:::: Simulation parameters :::

strategy : wgs
method : sample
genome : second_concat_recombined.fasta
prefix : wt_second_sample
id-prefix : S
depth : 20.000000
length-mean : (sample FASTQ)
length-sd : (sample FASTQ)
length-min : 100
length-max : 1000000
difference-ratio : 6:55:39
seed : 1754423015
sample : /FastData/kwilhoit/recombination_testing/all_barcode02_reads.fastq
sample-profile-id : (null)
accuracy-mean : (sample FASTQ)
accuracy-sd : (sample FASTQ)
accuracy-min : 0.750000
accuracy-max : 1.000000
pass_num : 1
hp-del-bias : 1.000000

:::: sample reads stats ::::

file name : /FastData/kwilhoit/recombination_testing/all_barcode02_reads.fastq

:: all reads ::
read num. : 208504
read total length : 863712515
read min length : 182
read max length : 96714

:: filtered reads ::
read num. : 208504
read total length : 863712515
read min length : 182
read max length : 96714
read length mean (SD) : 4142.426596 (4270.648405)
read accuracy mean (SD) : 0.959773 (0.025296)

:::: Reference st

In [1]:

%%bash
# Using a sample fastq file from real data to simulate reads (recommended)

pbsim --strategy wgs --method sample --sample /FastData/kwilhoit/recombination_testing/all_barcode02_reads.fastq --depth 20 --genome unrecombind_genome.fasta --prefix wt_unrecombined_sample

cat wt_unrecombined_*.fastq > all_wt_unrecombined_sample_simulated_reads.fastq

:::: Simulation parameters :::

strategy : wgs
method : sample
genome : unrecombind_genome.fasta
prefix : wt_unrecombined_sample
id-prefix : S
depth : 20.000000
length-mean : (sample FASTQ)
length-sd : (sample FASTQ)
length-min : 100
length-max : 1000000
difference-ratio : 6:55:39
seed : 1754420915
sample : /FastData/kwilhoit/recombination_testing/all_barcode02_reads.fastq
sample-profile-id : (null)
accuracy-mean : (sample FASTQ)
accuracy-sd : (sample FASTQ)
accuracy-min : 0.750000
accuracy-max : 1.000000
pass_num : 1
hp-del-bias : 1.000000

:::: sample reads stats ::::

file name : /FastData/kwilhoit/recombination_testing/all_barcode02_reads.fastq

:: all reads ::
read num. : 208504
read total length : 863712515
read min length : 182
read max length : 96714

:: filtered reads ::
read num. : 208504
read total length : 863712515
read min length : 182
read max length : 96714
read length mean (SD) : 4142.426596 (4270.648405)
read accuracy mean (SD) : 0.959773 (0.025296)

:::: Reference st

# Aligning simulated reads

In [ ]:
'''
minimap2 -ax  map-ont --secondary=no ../alignment_syri/genomes/H9920_sorted.fasta /FastData/kwilhoit/recombination_testing/second_computed_genomes/pbsim_sample_run/all_simulated_reads_second.fastq | samtools view -bS - | samtools sort -o ../alignment_syri/alignments/simulated_H99_alignment_sorted.bam 

samtools index ../alignment_syri/alignments/simulated_H99_alignment_sorted.bam

samtools stats ../alignment_syri/alignments/simulated_H99_alignment_sorted.bam  | grep ^SN  | cut -f 2- > ../alignment_syri/alignments/simulated_H99_alignment_stats.tsv

samtools view -b -f 4 ../alignment_syri/alignments/simulated_H99_alignment_sorted.bam > ../alignment_syri/alignments/simulated_H99_alignment_unmapped.bam
'''

'''
minimap2 -ax  map-ont ../alignment_syri/genomes/H9920_sorted.fasta /FastData/kwilhoit/recombination_testing/second_computed_genomes/pbsim_sample_run/all_simulated_reads_second.fastq | samtools view -bS - | samtools sort -o ../alignment_syri/alignments/simulated_secondary_H99_alignment_sorted.bam 

bamtools filter -in ../alignment_syri/alignments/simulated_secondary_H99_alignment_sorted.bam  -out ../alignment_syri/alignments/simulated_secondary_filtered_H99_alignment_sorted.bam -isPrimaryAlignment true

bamtools filter -in ../alignment_syri/alignments/simulated_secondary_H99_alignment_sorted.bam  -out ../alignment_syri/alignments/simulated_secondary_filtered_quality_H99_alignment_sorted.bam -mapQuality '>3'

samtools index ../alignment_syri/alignments/simulated_secondary_filtered_H99_alignment_sorted.bam

samtools stats ../alignment_syri/alignments/simulated_secondary_filtered_H99_alignment_sorted.bam | grep ^SN  | cut -f 2- > ../alignment_syri/alignments/simulated_secondary_H99_alignment_stats.tsv
'''

'''
minimap2 -ax  map-ont --secondary=no -Y /FastData/kwilhoit/recombination_testing/lra/concat_H99_Bt63.fasta  /FastData/kwilhoit/recombination_testing/second_computed_genomes/pbsim_sample_run/all_simulated_reads_second.fastq | samtools view -bS - | samtools sort -o ../alignment_syri/alignments/simulated_concat_alignment_sorted.bam 

samtools index ../alignment_syri/alignments/simulated_concat_alignment_sorted.bam

samtools stats ../alignment_syri/alignments/simulated_concat_alignment_sorted.bam  | grep ^SN  | cut -f 2- > ../alignment_syri/alignments/simulated_concat_alignment_stats.tsv

'''

'''
minimap2 -ax  map-ont -Y /FastData/kwilhoit/recombination_testing/lra//concat_H99_Bt63.fasta  /FastData/kwilhoit/recombination_testing/second_computed_genomes/pbsim_sample_run/all_simulated_reads_second.fastq | samtools view -bS - | samtools sort -o ../alignment_syri/alignments/simulated_concat_secondary_alignment_sorted.bam 

samtools index ../alignment_syri/alignments/simulated_concat_secondary_alignment_sorted.bam

samtools stats ../alignment_syri/alignments/simulated_concat_secondary_alignment_sorted.bam  | grep ^SN  | cut -f 2- > ../alignment_syri/alignments/simulated_concat_secondary_alignment_stats.tsv
'''

In [2]:
# Checking unmapped reads location in concatenated alignment

'''
samtools view -b -f 4 ../alignment_syri/alignments/simulated_H99_alignment_sorted.bam > ../alignment_syri/alignments/simulated_H99_alignment_unmapped.bam

grep -f <(samtools view ../alignment_syri/alignments/simulated_H99_alignment_unmapped.bam | cut -f 1) <(samtools view ../alignment_syri/alignments/simulated_concat_alignment_sorted.bam ) > ../alignment_syri/alignments/simulated_H99_unmapped_in_concat.sam

# Count number of Bt63_* mapped reads
grep -c Bt63_ ../alignment_syri/alignments/simulated_H99_unmapped_in_concat.sam

# Count number of H99_* mapped reads
grep -c H99_ ../alignment_syri/alignments/simulated_H99_unmapped_in_concat.sam

# Count number of unique read names in sam file
cut -f 1 ../alignment_syri/alignments/simulated_H99_unmapped_in_concat.sam | sort | uniq | wc -l
'''


''

In [ ]:
# Sorting and concatenating the reference genomes
'''
cat H9920_sorted.fasta Bt63.fasta > concat_H99_Bt_refs.fasta

cat recombined_genome2_sorted.fasta recombined_genome1.fasta > concat_r2_r1_recomb.fasta
'''

In [ ]:
from Bio import SeqIO

# Read the genome from the fasta file
genome = list(SeqIO.parse("concat_r1_r2_recomb.fasta", "fasta"))

# Rename the headers
for i, record in enumerate(genome, start=1):
    record.id = f"chr{str(i).zfill(2)}"
    record.description = ''

# Save the renamed genome to a file
SeqIO.write(genome, "renamed_concat_r1_r2_recomb.fasta", "fasta")

# Read the genome from the fasta file
genome = list(SeqIO.parse("concat_H99_Bt_refs.fasta", "fasta"))

# Rename the headers
for i, record in enumerate(genome, start=1):
    record.id = f"chr{str(i).zfill(2)}"
    record.description = ''

# Save the renamed genome to a file
SeqIO.write(genome, "renamed_concat_H99_Bt_refs.fasta", "fasta")



In [ ]:
# Checking recombination events by aligning concatenated recombined genome to the concatenated original genomes and plotting
'''
conda activate wfmash_env

samtools faidx renamed_concat_r1_r2_recomb.fasta

samtools faidx renamed_concat_H99_Bt_refs.fasta 

wfmash -t 8 renamed_concat_H99_Bt_refs.fasta renamed_concat_r1_r2_recomb.fasta > concated_refs_wfmash.paf

conda activate syri_env

syri -c concated_refs_wfmash.paf -r renamed_concat_H99_Bt_refs.fasta -q renamed_concat_r1_r2_recomb.fasta  -f -k -F P --dir concated_syri_out --prefix concated_

./plot_syri.sh 
'''